# Modelling decisions from experience

In [24]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, LSTM, GRU, SimpleRNN, Embedding, Conv1D, MaxPooling1D
from keras import optimizers,regularizers
from sklearn.preprocessing import MinMaxScaler

In [25]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

### Gathering the dataset

In [26]:
dataset = pd.read_csv('data/estimation.csv')
dataset_comp = pd.read_csv('data/competition.csv')

In [16]:
X_train = dataset.iloc[:, 0:-1].values
y_train = dataset.iloc[:, -1].values
X_test = dataset_comp.iloc[:, 0:-1].values
y_test = dataset_comp.iloc[:, -1].values

In [17]:
def normalize_columns(X):
	outcomes = [i for i in xrange(1, len(X[0]), 2)]
	#X[:, outcomes] = minmax_scale(X[:, outcomes])
	#X[:, outcomes] = minmax_scale(X[:, outcomes])
	mini = np.min(X)
	maxi = np.max(X)
	X[:, outcomes] -= mini
	X[:, outcomes] /=(maxi-mini)

In [19]:
#normalize_columns(X_train)
#normalize_columns(X_test)

# input_length = X_train.shape[1]/2
input_length = 8

### Preprocessing

In [20]:
# for efficient log-loss computation
y_train -= 1  
y_test -= 1  

In [21]:
# to feed the dataset as timesteps required for the LSTM training
X_train = np.reshape(X_train, (X_train.shape[0], input_length, X_train.shape[1]//input_length))
X_test = np.reshape(X_test, (X_test.shape[0], input_length, X_test.shape[1]//input_length))

In [23]:
X_train[0]

array([[ 1. , -0.3,  2. , -0.3,  1. , -0.3,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. 

### Basic sequential model

In [10]:
model = Sequential()
model.add(LSTM(200, return_sequences=False, kernel_initializer='normal', input_shape=(None, X_train.shape[2])))
model.add(Dropout(0.2))
# model.add(LSTM(200, kernel_initializer='normal'))
# model.add(Dropout(0.2))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

In [11]:
opt = optimizers.Adam(lr=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200)               204000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 204,201
Trainable params: 204,201
Non-trainable params: 0
_________________________________________________________________


### Fit the model

In [13]:
#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=500, batch_size=20)

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

### Sensitivity Analysis

#### No. of epochs

In [15]:
outfile = open('results/LSTM/diff-no-epochs.txt', 'w')
no_epochs = [10,50,100,200,300]

for epochs in no_epochs:
    model = Sequential()
    model.add(LSTM(100, kernel_initializer='normal', input_shape=(None, X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

    opt = optimizers.Adam()
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=20)
    
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    scores1 = model.evaluate(X_train, y_train, verbose=0)
    print("Accuracy: %.2f%%" % (scores1[1]*100))
    outfile.write(str((epochs, scores[1]*100, scores1[1]*100)))
    outfile.write('\n')
outfile.close()

Train on 1170 samples, validate on 1198 samples
Epoch 1/10
1170/1170 [==============================] - 2s 2ms/step - loss: 0.6854 - acc: 0.5598 - val_loss: 0.6903 - val_acc: 0.5225
Epoch 2/10
1170/1170 [==============================] - 1s 1ms/step - loss: 0.6723 - acc: 0.5889 - val_loss: 0.6847 - val_acc: 0.5501
Epoch 3/10
1170/1170 [==============================] - 1s 982us/step - loss: 0.6654 - acc: 0.6000 - val_loss: 0.6977 - val_acc: 0.5100
Epoch 4/10
1170/1170 [==============================] - 1s 991us/step - loss: 0.6607 - acc: 0.6051 - val_loss: 0.6806 - val_acc: 0.5651
Epoch 5/10
1170/1170 [==============================] - 1s 994us/step - loss: 0.6524 - acc: 0.6214 - val_loss: 0.7145 - val_acc: 0.5234
Epoch 6/10
1170/1170 [==============================] - 1s 1ms/step - loss: 0.6553 - acc: 0.6094 - val_loss: 0.6875 - val_acc: 0.5442
Epoch 7/10
1170/1170 [==============================] - 1s 1ms/step - loss: 0.6432 - acc: 0.6376 - val_loss: 0.6898 - val_acc: 0.5509
Epoch 8/

1170/1170 [==============================] - 1s 1ms/step - loss: 0.2996 - acc: 0.8504 - val_loss: 1.4499 - val_acc: 0.5927
Epoch 60/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2791 - acc: 0.8709 - val_loss: 1.3762 - val_acc: 0.5927
Epoch 61/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2601 - acc: 0.8778 - val_loss: 1.5192 - val_acc: 0.5868
Epoch 62/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2633 - acc: 0.8855 - val_loss: 1.4547 - val_acc: 0.5918
Epoch 63/100
1170/1170 [==============================] - 1s 992us/step - loss: 0.2643 - acc: 0.8863 - val_loss: 1.3909 - val_acc: 0.5952
Epoch 64/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2313 - acc: 0.8880 - val_loss: 1.5007 - val_acc: 0.5952
Epoch 65/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2130 - acc: 0.9051 - val_loss: 1.6890 - val_acc: 0.5793
Epoch 66/100
1170/1170 [==============================] - 1

1170/1170 [==============================] - 1s 1ms/step - loss: 0.1932 - acc: 0.9111 - val_loss: 1.7288 - val_acc: 0.5968
Epoch 79/200
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2241 - acc: 0.9009 - val_loss: 1.5804 - val_acc: 0.5985
Epoch 80/200
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2200 - acc: 0.9034 - val_loss: 1.6815 - val_acc: 0.6035
Epoch 81/200
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2022 - acc: 0.9060 - val_loss: 1.6538 - val_acc: 0.5902
Epoch 82/200
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1929 - acc: 0.9179 - val_loss: 1.7856 - val_acc: 0.5918
Epoch 83/200
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2084 - acc: 0.9103 - val_loss: 1.7444 - val_acc: 0.5927
Epoch 84/200
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1819 - acc: 0.9205 - val_loss: 1.9304 - val_acc: 0.5743
Epoch 85/200
1170/1170 [==============================] - 1s 

Epoch 138/200
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2226 - acc: 0.9060 - val_loss: 1.5859 - val_acc: 0.5985
Epoch 139/200
1170/1170 [==============================] - 1s 989us/step - loss: 0.1713 - acc: 0.9325 - val_loss: 1.7385 - val_acc: 0.5943
Epoch 140/200
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1813 - acc: 0.9222 - val_loss: 1.7593 - val_acc: 0.6002
Epoch 141/200
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1739 - acc: 0.9282 - val_loss: 1.7113 - val_acc: 0.5876
Epoch 142/200
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1704 - acc: 0.9316 - val_loss: 1.8037 - val_acc: 0.5927
Epoch 143/200
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1620 - acc: 0.9274 - val_loss: 1.8948 - val_acc: 0.5785
Epoch 144/200
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1901 - acc: 0.9265 - val_loss: 1.6774 - val_acc: 0.5993
Epoch 145/200
1170/1170 [==============

Epoch 198/200
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1245 - acc: 0.9419 - val_loss: 1.9437 - val_acc: 0.5835
Epoch 199/200
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1256 - acc: 0.9436 - val_loss: 2.0152 - val_acc: 0.6102
Epoch 200/200
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1111 - acc: 0.9513 - val_loss: 1.9464 - val_acc: 0.6135
Accuracy: 61.35%
Accuracy: 94.96%
Train on 1170 samples, validate on 1198 samples
Epoch 1/300
1170/1170 [==============================] - 3s 2ms/step - loss: 0.6851 - acc: 0.5744 - val_loss: 0.6970 - val_acc: 0.5184
Epoch 2/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.6735 - acc: 0.5906 - val_loss: 0.6964 - val_acc: 0.5209
Epoch 3/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.6671 - acc: 0.5974 - val_loss: 0.6895 - val_acc: 0.5392
Epoch 4/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.6665 - acc: 0.6051 - va

1170/1170 [==============================] - 1s 1ms/step - loss: 0.1474 - acc: 0.9333 - val_loss: 2.0907 - val_acc: 0.5985
Epoch 118/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1589 - acc: 0.9299 - val_loss: 2.1369 - val_acc: 0.5918
Epoch 119/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1635 - acc: 0.9265 - val_loss: 2.1017 - val_acc: 0.5935
Epoch 120/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1625 - acc: 0.9274 - val_loss: 2.0598 - val_acc: 0.5910
Epoch 121/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1486 - acc: 0.9325 - val_loss: 2.1875 - val_acc: 0.5868
Epoch 122/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1429 - acc: 0.9333 - val_loss: 2.2026 - val_acc: 0.5818
Epoch 123/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1534 - acc: 0.9393 - val_loss: 2.2576 - val_acc: 0.5893
Epoch 124/300
1170/1170 [==============================

Epoch 177/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1348 - acc: 0.9419 - val_loss: 2.1456 - val_acc: 0.5885
Epoch 178/300
1170/1170 [==============================] - 1s 958us/step - loss: 0.1651 - acc: 0.9333 - val_loss: 1.8143 - val_acc: 0.5993
Epoch 179/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1350 - acc: 0.9444 - val_loss: 1.9137 - val_acc: 0.5985
Epoch 180/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1315 - acc: 0.9444 - val_loss: 1.9640 - val_acc: 0.5977
Epoch 181/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1318 - acc: 0.9419 - val_loss: 2.0022 - val_acc: 0.5993
Epoch 182/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2032 - acc: 0.9197 - val_loss: 1.6892 - val_acc: 0.5860
Epoch 183/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1437 - acc: 0.9402 - val_loss: 1.7391 - val_acc: 0.5918
Epoch 184/300
1170/1170 [==============

Epoch 237/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1149 - acc: 0.9513 - val_loss: 2.0698 - val_acc: 0.6119
Epoch 238/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1125 - acc: 0.9513 - val_loss: 2.2374 - val_acc: 0.5927
Epoch 239/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1119 - acc: 0.9504 - val_loss: 2.2876 - val_acc: 0.5952
Epoch 240/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.0983 - acc: 0.9573 - val_loss: 2.3027 - val_acc: 0.6052
Epoch 241/300
1170/1170 [==============================] - 1s 984us/step - loss: 0.0943 - acc: 0.9598 - val_loss: 2.3557 - val_acc: 0.6035
Epoch 242/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1077 - acc: 0.9556 - val_loss: 2.3494 - val_acc: 0.5876
Epoch 243/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1604 - acc: 0.9299 - val_loss: 2.0475 - val_acc: 0.6002
Epoch 244/300
1170/1170 [==============

Epoch 297/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1162 - acc: 0.9504 - val_loss: 2.1721 - val_acc: 0.5935
Epoch 298/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.1196 - acc: 0.9521 - val_loss: 2.0819 - val_acc: 0.5902
Epoch 299/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.0847 - acc: 0.9581 - val_loss: 2.3168 - val_acc: 0.6010
Epoch 300/300
1170/1170 [==============================] - 1s 1ms/step - loss: 0.0771 - acc: 0.9641 - val_loss: 2.5595 - val_acc: 0.6093
Accuracy: 60.93%
Accuracy: 96.75%


#### Batch size

In [14]:
outfile = open('results/LSTM/diff-no-bs.txt', 'w')
no_bs = [10,20,30,50,80]

for bs in no_bs:
    model = Sequential()
    model.add(LSTM(100, kernel_initializer='normal', input_shape=(None, X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

    opt = optimizers.Adam(lr=0.01)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=bs)
    
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    scores1 = model.evaluate(X_train, y_train, verbose=0)
    print("Accuracy: %.2f%%" % (scores1[1]*100))
    outfile.write(str((bs, scores[1]*100, scores1[1]*100)))
    outfile.write('\n')
outfile.close()

Train on 1170 samples, validate on 1198 samples
Epoch 1/100
1170/1170 [==============================] - 3s 3ms/step - loss: 0.6938 - acc: 0.5419 - val_loss: 0.7151 - val_acc: 0.4816
Epoch 2/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.6942 - acc: 0.5419 - val_loss: 0.6885 - val_acc: 0.5134
Epoch 3/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.6844 - acc: 0.5786 - val_loss: 0.7004 - val_acc: 0.5209
Epoch 4/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.6779 - acc: 0.5829 - val_loss: 0.6918 - val_acc: 0.5275
Epoch 5/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.6748 - acc: 0.5940 - val_loss: 0.7195 - val_acc: 0.5083
Epoch 6/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.6759 - acc: 0.5906 - val_loss: 0.6999 - val_acc: 0.5209
Epoch 7/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.6706 - acc: 0.5880 - val_loss: 0.6863 - val_acc: 0.5601
Epoch 8

Epoch 61/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.5375 - acc: 0.7051 - val_loss: 0.8712 - val_acc: 0.5551
Epoch 62/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.5318 - acc: 0.7103 - val_loss: 0.9377 - val_acc: 0.5559
Epoch 63/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.5298 - acc: 0.7171 - val_loss: 0.8736 - val_acc: 0.5868
Epoch 64/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.5224 - acc: 0.7094 - val_loss: 0.9583 - val_acc: 0.5467
Epoch 65/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.5062 - acc: 0.7325 - val_loss: 1.0445 - val_acc: 0.5409
Epoch 66/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.5113 - acc: 0.7308 - val_loss: 0.9296 - val_acc: 0.5676
Epoch 67/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.4980 - acc: 0.7171 - val_loss: 1.0084 - val_acc: 0.5551
Epoch 68/100
1170/1170 [========================

1170/1170 [==============================] - 1s 945us/step - loss: 0.4717 - acc: 0.7513 - val_loss: 1.0326 - val_acc: 0.5392
Epoch 81/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.4648 - acc: 0.7487 - val_loss: 1.0104 - val_acc: 0.5518
Epoch 82/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.4487 - acc: 0.7761 - val_loss: 1.0894 - val_acc: 0.5484
Epoch 83/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.4845 - acc: 0.7419 - val_loss: 0.9995 - val_acc: 0.5492
Epoch 84/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.4682 - acc: 0.7325 - val_loss: 1.1374 - val_acc: 0.5760
Epoch 85/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.4566 - acc: 0.7607 - val_loss: 1.0695 - val_acc: 0.5810
Epoch 86/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.4524 - acc: 0.7598 - val_loss: 1.1631 - val_acc: 0.5442
Epoch 87/100
1170/1170 [==============================] - 1

1170/1170 [==============================] - 1s 722us/step - loss: 0.3516 - acc: 0.8419 - val_loss: 1.3534 - val_acc: 0.5710
Epoch 99/100
1170/1170 [==============================] - 1s 748us/step - loss: 0.3630 - acc: 0.8359 - val_loss: 1.3475 - val_acc: 0.5626
Epoch 100/100
1170/1170 [==============================] - 1s 784us/step - loss: 0.3517 - acc: 0.8470 - val_loss: 1.3690 - val_acc: 0.5735
Accuracy: 57.35%
Accuracy: 85.98%
Train on 1170 samples, validate on 1198 samples
Epoch 1/100
1170/1170 [==============================] - 2s 2ms/step - loss: 0.6832 - acc: 0.5701 - val_loss: 0.7007 - val_acc: 0.5250
Epoch 2/100
1170/1170 [==============================] - 1s 483us/step - loss: 0.6753 - acc: 0.5795 - val_loss: 0.6873 - val_acc: 0.5501
Epoch 3/100
1170/1170 [==============================] - 1s 461us/step - loss: 0.6719 - acc: 0.6077 - val_loss: 0.6883 - val_acc: 0.5234
Epoch 4/100
1170/1170 [==============================] - 0s 391us/step - loss: 0.6712 - acc: 0.5769 - val_l

Epoch 15/100
1170/1170 [==============================] - 0s 332us/step - loss: 0.6088 - acc: 0.6803 - val_loss: 0.7686 - val_acc: 0.5401
Epoch 16/100
1170/1170 [==============================] - 0s 321us/step - loss: 0.5887 - acc: 0.6957 - val_loss: 0.7702 - val_acc: 0.5568
Epoch 17/100
1170/1170 [==============================] - 0s 323us/step - loss: 0.5978 - acc: 0.6846 - val_loss: 0.7724 - val_acc: 0.5676
Epoch 18/100
1170/1170 [==============================] - 0s 336us/step - loss: 0.5690 - acc: 0.7060 - val_loss: 0.7548 - val_acc: 0.5426
Epoch 19/100
1170/1170 [==============================] - 0s 342us/step - loss: 0.5643 - acc: 0.6983 - val_loss: 0.8027 - val_acc: 0.5710
Epoch 20/100
1170/1170 [==============================] - 0s 332us/step - loss: 0.5581 - acc: 0.7111 - val_loss: 0.8174 - val_acc: 0.5501
Epoch 21/100
1170/1170 [==============================] - 0s 326us/step - loss: 0.5449 - acc: 0.7197 - val_loss: 0.8843 - val_acc: 0.5409
Epoch 22/100
1170/1170 [==========

#### No. of LSTM units

In [16]:
outfile = open('results/LSTM/diff-no-units.txt', 'w')
no_units = [1,10,50,100,500]

for units in no_units:
    model = Sequential()
    model.add(LSTM(units, kernel_initializer='normal', input_shape=(None, X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

    opt = optimizers.Adam(lr=0.01)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=20)
    
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    scores1 = model.evaluate(X_train, y_train, verbose=0)
    print("Accuracy: %.2f%%" % (scores1[1]*100))
    outfile.write(str((units, scores[1]*100, scores1[1]*100)))
    outfile.write('\n')
outfile.close()

Train on 1170 samples, validate on 1198 samples
Epoch 1/100
1170/1170 [==============================] - 3s 2ms/step - loss: 0.6931 - acc: 0.5034 - val_loss: 0.6908 - val_acc: 0.5543
Epoch 2/100
1170/1170 [==============================] - 1s 933us/step - loss: 0.6923 - acc: 0.4991 - val_loss: 0.6878 - val_acc: 0.5526
Epoch 3/100
1170/1170 [==============================] - 1s 949us/step - loss: 0.6867 - acc: 0.5427 - val_loss: 0.6879 - val_acc: 0.5534
Epoch 4/100
1170/1170 [==============================] - 1s 946us/step - loss: 0.6859 - acc: 0.5453 - val_loss: 0.6913 - val_acc: 0.5242
Epoch 5/100
1170/1170 [==============================] - 1s 960us/step - loss: 0.6795 - acc: 0.5675 - val_loss: 0.6939 - val_acc: 0.5242
Epoch 6/100
1170/1170 [==============================] - 1s 962us/step - loss: 0.6775 - acc: 0.5761 - val_loss: 0.6947 - val_acc: 0.5259
Epoch 7/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.6771 - acc: 0.5829 - val_loss: 0.6894 - val_acc: 0.55

1170/1170 [==============================] - 1s 960us/step - loss: 0.6145 - acc: 0.6658 - val_loss: 0.7403 - val_acc: 0.5492
Epoch 20/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.6184 - acc: 0.6590 - val_loss: 0.7505 - val_acc: 0.5484
Epoch 21/100
1170/1170 [==============================] - 1s 967us/step - loss: 0.6134 - acc: 0.6462 - val_loss: 0.7495 - val_acc: 0.5551
Epoch 22/100
1170/1170 [==============================] - 1s 947us/step - loss: 0.6076 - acc: 0.6607 - val_loss: 0.7324 - val_acc: 0.5518
Epoch 23/100
1170/1170 [==============================] - 1s 949us/step - loss: 0.5864 - acc: 0.6692 - val_loss: 0.8162 - val_acc: 0.5367
Epoch 24/100
1170/1170 [==============================] - 1s 993us/step - loss: 0.6059 - acc: 0.6641 - val_loss: 0.7839 - val_acc: 0.5476
Epoch 25/100
1170/1170 [==============================] - 1s 976us/step - loss: 0.5955 - acc: 0.6752 - val_loss: 0.7627 - val_acc: 0.5209
Epoch 26/100
1170/1170 [=========================

1170/1170 [==============================] - 1s 1ms/step - loss: 0.5716 - acc: 0.6744 - val_loss: 0.8428 - val_acc: 0.5543
Epoch 38/100
1170/1170 [==============================] - 1s 998us/step - loss: 0.5640 - acc: 0.6778 - val_loss: 0.8140 - val_acc: 0.5601
Epoch 39/100
1170/1170 [==============================] - 1s 992us/step - loss: 0.5716 - acc: 0.6786 - val_loss: 0.8368 - val_acc: 0.5317
Epoch 40/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.5509 - acc: 0.7000 - val_loss: 0.8779 - val_acc: 0.5426
Epoch 41/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.5527 - acc: 0.6872 - val_loss: 0.8560 - val_acc: 0.5134
Epoch 42/100
1170/1170 [==============================] - 1s 978us/step - loss: 0.5473 - acc: 0.7043 - val_loss: 0.8503 - val_acc: 0.5559
Epoch 43/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.5367 - acc: 0.7017 - val_loss: 1.0228 - val_acc: 0.5668
Epoch 44/100
1170/1170 [==============================]

Epoch 97/100
1170/1170 [==============================] - 1s 975us/step - loss: 0.4582 - acc: 0.7470 - val_loss: 1.2465 - val_acc: 0.5484
Epoch 98/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.4660 - acc: 0.7761 - val_loss: 0.9940 - val_acc: 0.5693
Epoch 99/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.4386 - acc: 0.7744 - val_loss: 1.2073 - val_acc: 0.5668
Epoch 100/100
1170/1170 [==============================] - 1s 938us/step - loss: 0.4407 - acc: 0.7615 - val_loss: 1.1688 - val_acc: 0.5476
Accuracy: 54.76%
Accuracy: 79.40%
Train on 1170 samples, validate on 1198 samples
Epoch 1/100
1170/1170 [==============================] - 4s 3ms/step - loss: 0.6860 - acc: 0.5470 - val_loss: 0.6868 - val_acc: 0.5534
Epoch 2/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.6834 - acc: 0.5812 - val_loss: 0.7250 - val_acc: 0.5284
Epoch 3/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.6794 - acc: 0.5889 -

Epoch 16/100
1170/1170 [==============================] - 4s 4ms/step - loss: 0.6520 - acc: 0.6291 - val_loss: 0.7193 - val_acc: 0.5451
Epoch 17/100
1170/1170 [==============================] - 4s 4ms/step - loss: 0.6545 - acc: 0.6043 - val_loss: 0.7017 - val_acc: 0.5543
Epoch 18/100
1170/1170 [==============================] - 4s 4ms/step - loss: 0.6538 - acc: 0.6197 - val_loss: 0.7214 - val_acc: 0.5501
Epoch 19/100
1170/1170 [==============================] - 4s 4ms/step - loss: 0.6560 - acc: 0.5949 - val_loss: 0.7207 - val_acc: 0.5050
Epoch 20/100
1170/1170 [==============================] - 4s 4ms/step - loss: 0.6461 - acc: 0.6068 - val_loss: 0.7084 - val_acc: 0.5376
Epoch 21/100
1170/1170 [==============================] - 4s 4ms/step - loss: 0.6216 - acc: 0.6291 - val_loss: 0.8083 - val_acc: 0.5267
Epoch 22/100
1170/1170 [==============================] - 4s 4ms/step - loss: 0.6507 - acc: 0.6222 - val_loss: 0.7040 - val_acc: 0.5209
Epoch 23/100
1170/1170 [========================

#### No. of layers

In [19]:
# %%script false
outfile = open('results/LSTM/diff-no-layers.txt', 'w')
no_layers = [1,2,3,5,7]

d = 0.2

for layers in no_layers:
    model = Sequential()
    if(layers == 1):
        model.add(LSTM(100, kernel_initializer='normal', input_shape=(None, X_train.shape[2])))
    else:
        model.add(LSTM(100, kernel_initializer='normal', input_shape=(None, X_train.shape[2]), return_sequences=True))
        while(layers-2):
            model.add(LSTM(100, kernel_initializer='normal', input_shape=(None, input_length), return_sequences=True))
            layers -= 1
        model.add(LSTM(100, kernel_initializer='normal', input_shape=(None, input_length)))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

    opt = optimizers.Adam()
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=20)
    
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    scores1 = model.evaluate(X_train, y_train, verbose=0)
    print("Accuracy: %.2f%%" % (scores1[1]*100))
    outfile.write(str((layers, scores[1]*100, scores1[1]*100)))
    outfile.write('\n')
outfile.close()

Train on 1170 samples, validate on 1198 samples
Epoch 1/100
1170/1170 [==============================] - 4s 3ms/step - loss: 0.6862 - acc: 0.5692 - val_loss: 0.6887 - val_acc: 0.5175
Epoch 2/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.6734 - acc: 0.5906 - val_loss: 0.6921 - val_acc: 0.5050
Epoch 3/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.6678 - acc: 0.5940 - val_loss: 0.6886 - val_acc: 0.5125
Epoch 4/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.6614 - acc: 0.6094 - val_loss: 0.6891 - val_acc: 0.5217
Epoch 5/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.6587 - acc: 0.6103 - val_loss: 0.6977 - val_acc: 0.5551
Epoch 6/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.6471 - acc: 0.6342 - val_loss: 0.6876 - val_acc: 0.5451
Epoch 7/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.6406 - acc: 0.6316 - val_loss: 0.7060 - val_acc: 0.5267
Epoch 8

Epoch 61/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2449 - acc: 0.8915 - val_loss: 1.4328 - val_acc: 0.5993
Epoch 62/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2390 - acc: 0.8949 - val_loss: 1.5166 - val_acc: 0.5885
Epoch 63/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2383 - acc: 0.8897 - val_loss: 1.5562 - val_acc: 0.5935
Epoch 64/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2415 - acc: 0.8949 - val_loss: 1.5412 - val_acc: 0.5718
Epoch 65/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2428 - acc: 0.8897 - val_loss: 1.5762 - val_acc: 0.5776
Epoch 66/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2615 - acc: 0.8846 - val_loss: 1.4421 - val_acc: 0.5718
Epoch 67/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2252 - acc: 0.8974 - val_loss: 1.6801 - val_acc: 0.5785
Epoch 68/100
1170/1170 [========================

KeyboardInterrupt: 

#### Diff. loss

In [ ]:
outfile = open('results/LSTM/diff-no-loss.txt', 'w')
losses = ['mse','mae','binary_crossentropy']

for loss in losses:
    model = Sequential()
    model.add(LSTM(100, kernel_initializer='normal', input_shape=(None, X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

    opt = optimizers.Adam(lr=0.01)
    model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=20)
    
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    scores1 = model.evaluate(X_train, y_train, verbose=0)
    print("Accuracy: %.2f%%" % (scores1[1]*100))
    outfile.write(str((loss, scores[1]*100, scores1[1]*100)))
    outfile.write('\n')
outfile.close()

Train on 1170 samples, validate on 1198 samples
Epoch 1/100
1170/1170 [==============================] - 5s 4ms/step - loss: 0.2468 - acc: 0.5556 - val_loss: 0.2509 - val_acc: 0.4624
Epoch 2/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2451 - acc: 0.5692 - val_loss: 0.2451 - val_acc: 0.5601
Epoch 3/100
1170/1170 [==============================] - 1s 1ms/step - loss: 0.2423 - acc: 0.5709 - val_loss: 0.2500 - val_acc: 0.5292
Epoch 4/100
1170/1170 [==============================] - 1s 915us/step - loss: 0.2420 - acc: 0.5821 - val_loss: 0.2464 - val_acc: 0.5367
Epoch 5/100
1170/1170 [==============================] - 1s 886us/step - loss: 0.2394 - acc: 0.6034 - val_loss: 0.2471 - val_acc: 0.5526
Epoch 6/100
1170/1170 [==============================] - 1s 842us/step - loss: 0.2366 - acc: 0.5940 - val_loss: 0.2458 - val_acc: 0.5409
Epoch 7/100
1170/1170 [==============================] - 1s 843us/step - loss: 0.2343 - acc: 0.6188 - val_loss: 0.2598 - val_acc: 0.5209